Detected Jupyter notebook. Loading juliacall extension. Set `PYSR_AUTOLOAD_EXTENSIONS=no` to disable.


In [ ]:
!pip install google-auth
from google.colab import auth
auth.authenticate_user()
from google.colab import drive
drive.mount('/content/drive')
!ls /content/drive/MyDrive
import os

path = '/content/drive/My Drive'

if os.path.isdir(path):
    contents = os.listdir(path)
    print(f"Contents of {path}:")
    for item in contents:
        print(item)

files = os.listdir('/content/drive/My Drive/bcg_dataset')
files = os.listdir('/content/drive/My Drive/SP_BCG')
files = os.listdir('/content/drive/My Drive/DP_BCG')
print(files)

Mounted at /content/drive
 all_feature_importances_SP.csv			     predictions_fold_0.csv
 average_model_performance_results_DP_Original.csv   predictions_fold_1.csv
 average_model_performance_results_Original.csv      predictions_fold_2.csv
 average_model_performance_results_SP_SR.csv	     predictions_fold_3.csv
 bcg_dataset					     predictions_fold_4.csv
'Colab Notebooks'				     results_original_SP.csv
 DP_BCG						     results_selected_SP.csv
 model_performance_results_DP_Original.csv	     selected_feature_importances_SP.csv
 model_performance_results_SP_Original.csv	     SP_BCG
 model_performance_results_SR_SP.csv
Contents of /content/drive/My Drive:
bcg_dataset
Colab Notebooks
average_model_performance_results_SP_SR.csv
model_performance_results_SR_SP.csv
model_performance_results_SP_Original.csv
average_model_performance_results_Original.csv
predictions_fold_0.csv
predictions_fold_1.csv
predictions_fold_2.csv
predictions_fold_3.csv
predictions_fold_4.csv
model_performance_results

In [ ]:
!pip install pysr
import pandas as pd
import pysr
from pysr import PySRRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import numpy as np


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 734.1 kB/s eta 0:00:00
[juliapkg] Locating Julia ~1.6.7, ~1.7, ~1.8, ~1.9, =1.10.0
[juliapkg] Querying Julia versions from https://julialang-s3.julialang.org/bin/versions.json
[juliapkg] WARNING: About to install Julia 1.10.0 to /root/.julia/environments/pyjuliapkg/pyjuliapkg/install.
[juliapkg]   If you use juliapkg in more than one environment, you are likely to have Julia
[juliapkg]   installed in multiple locations. It is recommended to install JuliaUp
[juliapkg]   (https://github.com/JuliaLang/juliaup) or Julia (https://julialang.org/downloads)
[juliapkg]   yourself.
[juliapkg] Downloading Julia from https://julialang-s3.julialang.org/bin/linux/x64/1.10/julia-1.10.0-linux-x86_64.tar.gz
             download complete
[juliapkg] Verifying download
[juliapkg] Installing Julia 1.10.0 to /root/.julia/environments/pyjuliapkg/pyjuliapkg/install
[juliapkg] Using Julia 1.10.0 at /root/.julia/environments/pyjuliapkg/pyjuliapkg/insta

In [ ]:
def load_datasets():
    fold_files = [
    '/content/drive/My Drive/bcg_dataset/feat_fold_0.csv',
        '/content/drive/My Drive/bcg_dataset/feat_fold_1.csv',
        '/content/drive/My Drive/bcg_dataset/feat_fold_2.csv',
        '/content/drive/My Drive/bcg_dataset/feat_fold_3.csv',
        '/content/drive/My Drive/bcg_dataset/feat_fold_4.csv'
    ]
    return [pd.read_csv(f) for f in fold_files]


In [ ]:
def split_datasets(folds):
    num_folds = len(folds)
    splits = []
    for r in range( num_folds):  # From 1 to 4 folds for training
        train_indices = list(range(num_folds))

        train_indices.remove(r)
        test_indices = [r]
        train_dfs = [folds[i] for i in train_indices]
        test_dfs = [folds[i] for i in test_indices]
        splits.append((pd.concat(train_dfs), pd.concat(test_dfs)))
    return splits


In [ ]:
pysr_settings = {
'niterations':100,
    'timeout_in_seconds':60 * 60 * 24,
    'early_stop_condition':(
        "stop_if(loss, complexity) = loss < 1e-6 && complexity < 10"
        # Stop early if we find a good and simple equation
    ),
    'ncycles_per_iteration':200,
    'binary_operators':["+", "*", "/", "-"],
    'unary_operators':["sin", "cos", "exp", "log", "abs", "sqrt", "inv"],
    'extra_sympy_mappings':{'inv': lambda x: 1/x},
    #'populations':50,
    #'population_size':1000,
    #'max_evals':5000,
    #'maxdepth':10,
    #'maxsize':1000,
    'progress':True,
    #'precision':64,
    #'weight_randomize':0.1,
    #'cluster_manager':None,
    #'parsimony':0.001,
    #'procs':4,  # Set to 0 to turn off parallelism for determinism
    'model_selection':"best",
    'verbosity':0,
    #'select_k_features':29,
    #'select_k_features':20,
    #'complexity_of_constants':2,
    #'elementwise_loss':"L1DistLoss()",
    # random_state=42,  # Set a fixed seed for reproducibility
    #deterministic=True,  # Ensure deterministic behavior
    'warm_start':False,  # Start fresh for this example; set to True to continue from last .fit() call
    #'bumper':True,

}

In [ ]:
def perform_symbolic_regression(splits, pysr_settings, number_of_features = 10):

    results = []

    for i, (train_data, test_data) in enumerate(splits):
        # Prepare the datasets, excluding 'patient' and 'trial' columns for X
        X_train = train_data.drop(['SP', 'DP', 'trial', 'patient'], axis=1)
        y_train = train_data[['SP', 'DP']]
        X_test = test_data.drop(['SP', 'DP', 'trial', 'patient'], axis=1)
        y_test = test_data[['SP', 'DP']]

        for target, y_tr, y_te in zip(['SP', 'DP'], [y_train.iloc[:, 0], y_train.iloc[:, 1]], [y_test.iloc[:, 0], y_test.iloc[:, 1]]):

            estimator = LinearRegression() #RandomForestRegressor()#
            rfe = RFE(estimator=estimator,n_features_to_select=number_of_features, step=1)
            rfe.fit(X_train, y_tr)

            best_features = rfe.feature_names_in_[rfe.ranking_[0:number_of_features]]
            #print(best_features)

            model = PySRRegressor(**pysr_settings)
            model.fit(X_train[best_features], y_tr)

            predictions = model.predict(X_test[best_features])
            score = r2_score(y_te, predictions)
            mse = mean_squared_error(y_te, predictions)
            mse_naive = mean_squared_error(y_te, [np.mean(y_te)]*len(y_te))
            mae = mean_absolute_error(y_te, predictions)
            mae_naive = mean_absolute_error(y_te, [np.mean(y_te)]*len(y_te))

            print(f"Test R^2 score for {target} using split {i}: {score}")
            print(f"Test MSE for {target} using split {i}: {mse}")

            equations = model.get_best()
            best_equation = equations['equation']

            # Storing results
            results.append({
                'split': i,
                'target': target,
                'R2_score': score,
                'MSE': mse,
                'MMSE' : mse/mse_naive,
                'best_equation': best_equation,
                'MAE': mae,
                'MASE' : mae/mae_naive,
                'features' : best_features
            })

    return results



In [ ]:
def main():
    folds = load_datasets()
    splits = split_datasets(folds)
    results = perform_symbolic_regression(splits, pysr_settings)

    # Convert results to DataFrame for easier analysis and export
    results_df = pd.DataFrame(results)
    print(results_df)

if __name__ == "__main__":
    main()


/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1302: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1889: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(


Test R^2 score for SP using split 0: -0.3672688112222269
Test MSE for SP using split 0: 566.2222035699444


/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1302: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1889: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(


Test R^2 score for DP using split 0: -0.18927965952469372
Test MSE for DP using split 0: 129.0504493093141


/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1302: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1889: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(


...

Test R^2 score for DP using split 3: -1.488174114348936
Test MSE for DP using split 3: 114.17090444885437


/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1302: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1889: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(


Test R^2 score for SP using split 4: -0.38604337454462057
Test MSE for SP using split 4: 166.49151209710982


/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1302: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/pysr/sr.py:1889: UserWarning: Note: you are running with 10 features or more. Genetic algorithms like used in PySR scale poorly with large numbers of features. You should run PySR for more `niterations` to ensure it can find the correct variables, or, alternatively, do a dimensionality reduction beforehand. For example, `X = PCA(n_components=6).fit_transform(X)`, using scikit-learn's `PCA` class, will reduce the number of features to 6 in an interpretable way, as each resultant feature will be a linear combination of the original features. 
  warnings.warn(


Test R^2 score for DP using split 4: -0.06423876782666715
Test MSE for DP using split 4: 67.93013230768045
   split target  R2_score         MSE      MMSE  \
0      0     SP -0.367269  566.222204  1.367269   
1      0     DP -0.189280  129.050449  1.189280   
2      1     SP  0.014944  193.310030  0.985056   
3      1     DP -0.123574   92.046023  1.123574   
4      2     SP -0.051041  101.817688  1.051041   
5      2     DP -0.036568   63.292833  1.036568   
6      3     SP -1.024041  203.446930  2.024041   
7      3     DP -1.488174  114.170904  2.488174   
8      4     SP -0.386043  166.491512  1.386043   
9      4     DP -0.064239   67.930132  1.064239   

                                       best_equation        MAE      MASE  \
0                                     abs(118.28823)  17.277001  1.089003   
1                                      abs(-66.2375)   7.814936  0.931302   
2  (((T_c_norm * dsdc_16) - T_peak_c) * 210.86816...  10.450998  0.938755   
3                      